In [ ]:
import numpy as np
from pathlib import Path
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
import tensorflow as tf

In [ ]:
# Paths and configuration
MODEL_PATH = Path("models/alzheimer_model.h5")
HISTORY_PATH = Path("models/training_history.npy")
CLASSES = ["non_demented", "very_mild_dementia", "mild_dementia", "moderate_dementia"]

In [ ]:
# Load model and data
model = load_model(MODEL_PATH)
history = np.load(HISTORY_PATH, allow_pickle=True).item()

In [ ]:
# Load evaluation data
X_test = np.load(Path("models/X_test.npy")) / 255.0
y_test = np.load(Path("models/y_test.npy"))

In [ ]:
# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Predictions and evaluation metrics
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=CLASSES))

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=CLASSES, yticklabels=CLASSES)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# ROC-AUC Curves
plt.figure()
for i, cls in enumerate(CLASSES):
    fpr, tpr, _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{cls} (AUC = {roc_auc:.2f})")
plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [ ]:
# Grad-CAM Visualization
def grad_cam(model, img, class_idx, layer_name="block5_conv3"):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(np.expand_dims(img, axis=0))
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1).numpy()
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
# Test Grad-CAM on a sample image
sample_idx = 0  # Change this index to test different images
sample_image = X_test[sample_idx]
sample_class = y_true_classes[sample_idx]
heatmap = grad_cam(model, sample_image, sample_class)

plt.imshow(sample_image)
plt.imshow(heatmap, cmap="jet", alpha=0.5)
plt.title(f"Grad-CAM for Class: {CLASSES[sample_class]}")
plt.show()